<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_3/Supp_Fig_3abc/1_host_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Host matrix quality control (QC) and filtering

In [ ]:
!pip install -q kb_python gget scanpy

In [ ]:
import kb_python.utils as kb_utils
import numpy as np
from scipy.interpolate import interpn
import pandas as pd
import gget
import json
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
%config InlineBackend.figure_format='retina'

import scanpy as sc
sc.set_figure_params(figsize=(6, 6), frameon=False)
sc.settings.n_jobs=2

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

### Load data aligned to macaque and dog (for MDCK spike-ins) genomes:
The alignment was performed in [this notebook](https://github.com/pachterlab/LSCHWCP_2023/blob/main/Notebooks/align_macaque_PBMC_data/1_align_to_host.ipynb).

In [ ]:
# Load host alignment results from Caltech Data
# !wget
# !tar -xvf host_alignment_results.tar.gz
!unzip host_alignment_results.zip

In [ ]:
%%time
# Filepath to counts
X = "host_alignment_results/bustools_count/output.mtx"
# Filepath to barcode metadata
var_path = "host_alignment_results/bustools_count/output.genes.txt"
# Filepath to gene metadata
obs_path = "host_alignment_results/bustools_count/output.barcodes.txt"

# Create AnnData object
host_adata = kb_utils.import_matrix_as_anndata(X, obs_path, var_path)
host_adata

Since we used a batch file together with the `--batch-barcodes` option to align all fastq files simultaneously, we can use the batch barcodes to add information about which cells came from which batch/library:

In [ ]:
# Add sample barcodes to adata
sb_file = open("host_alignment_results/bustools_count/output.barcodes.prefix.txt")
sample_barcodes = sb_file.read().splitlines()
sb_file.close()

# Only use last 16 bases because sample barcode is always 16 bases in length
host_adata.obs["sample_barcode"] = [bc[-16:] for bc in sample_barcodes]

In [ ]:
# Create barcode to sample lookup
bc2sample_df = pd.DataFrame()

b_file = open("host_alignment_results/host_canine/matrix.sample.barcodes")
barcodes = b_file.read().splitlines()
b_file.close()

s_file = open("host_alignment_results/host_canine/matrix.cells")
samples = s_file.read().splitlines()
s_file.close()

bc2sample_df["sample_barcode"] = barcodes
bc2sample_df["srr"] = samples
bc2sample_df

In [ ]:
host_adata.obs["barcode"] = host_adata.obs.index.values
host_adata.obs = host_adata.obs.merge(bc2sample_df, on="sample_barcode", how="left").set_index("barcode", drop=False)
host_adata.obs

In [ ]:
# Remove version number from Ensembl IDs
host_adata.var["gene_id"] = host_adata.var.index.values
host_adata.var["ensembl_id"] = [id_.split(".")[0] for id_ in host_adata.var.index.values]
host_adata.var = host_adata.var.set_index("ensembl_id")
host_adata.var

___
# Filter host cells based on knee plot

In [ ]:
color = "grey"
filter_color = "lightgrey"
fontsize = 14

# Expected number of cells
expected_num_cells = 600000
# Minimum number of genes per cell
min_genes = 300

In [ ]:
def yex(ax, col="k", alpha=0.75):
    """
    Funciton to add linear graph to plot.
    Call after defining x and y scales
    """
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    # Plot both limits against each other
    ax.plot(lims, lims, col, alpha=alpha, zorder=1)
    ax.set(**{
        "aspect": "equal",
        "xlim": lims,
        "ylim": lims
    })
    return ax

In [ ]:
%%time
# Generate knee and library saturation plots
fig, axs = plt.subplots(
    figsize=(14, 7),
    ncols=2,
    constrained_layout=True,
)

adata = host_adata

## Plot knee plot
ax = axs[0]

knee = np.sort((np.array(adata.X.sum(axis=1))).flatten())[::-1]

# Create masks for cells that will be kept/filtered out based on minimum UMI count
min_umi = knee[expected_num_cells]
keep_mask = np.ma.masked_where(knee > min_umi, knee).mask
filter_mask = np.invert(keep_mask)

# Plot all cells in filter out color
ax.plot(knee, range(len(knee)), linewidth=5, color=filter_color)
# Overlay with cells to keep in dark color
ax.plot(knee[keep_mask], range(len(knee[keep_mask])), linewidth=5, color=color)
ax.set_xscale("log")
ax.set_yscale("log")

# Add lines to show filtering cut offs
ax.axvline(x=knee[expected_num_cells], linewidth=1.5, color="k", ls="--")
ax.axhline(y=expected_num_cells, linewidth=1.5, color="k", ls="--")

# Define axis labels and title
ax.set_xlabel("UMI Counts", fontsize=fontsize)
ax.set_ylabel("Set of Barcodes", fontsize=fontsize)
# ax.set_title(title, fontsize=fontsize + 2)

# Change fontsize of tick labels
ax.tick_params(axis="both", labelsize=fontsize)

# # Set x and y lim
ax.set_xlim(1, 10 ** 8)
ax.set_ylim(1, 10 ** 8)

# Add grid and set below graph
ax.grid(True, which="both", color="lightgray")
ax.set_axisbelow(True)

# Add invisible linear graph to match square library sat plot
yex(ax, alpha=0)


## Plot library saturation plot
ax = axs[1]

# Number of UMI counts per cell
x = np.asarray(adata.X.sum(axis=1))[:, 0]
# Number of different genes detected per cell
y = np.asarray(np.sum(adata.X > 0, axis=1))[:, 0]

# Create masks for cells that will be kept/filtered out based on minimum UMI count from knee plot
keep_mask = np.ma.masked_where(x > min_umi, x).mask
filter_mask = np.invert(keep_mask)

ax.scatter(x[keep_mask], y[keep_mask], alpha=0.5, color=color, zorder=2)
ax.scatter(x[filter_mask], y[filter_mask], alpha=0.5, color=filter_color, zorder=2)

# Add lines to show filtering cut off
ax.axvline(x=min_umi, linewidth=1.5, color="k", ls="--", zorder=3)

ax.set_xscale("log")
ax.set_yscale("log")

# Define axis labels and title
ax.set_xlabel("UMI Counts", fontsize=fontsize)
ax.set_ylabel("Genes Detected", fontsize=fontsize)
# ax.set_title(title, fontsize=fontsize+2)

# Change fontsize of tick labels
ax.tick_params(axis="both", labelsize=fontsize)

# # Set x and y lim
ax.set_xlim(1, 10 ** 5)
ax.set_ylim(1, 10 ** 5)

# Add grid and set below graph
ax.set_axisbelow(True)

# Add linear graph
yex(ax)

# Save figure
fig.savefig(
    f"host_QC_knee_+_library_sat.png", dpi=300, bbox_inches="tight"
)

fig.show()

In [ ]:
## Filter based on knee plot and minimum number of genes
print("adata (before filtering): ", host_adata)
print(f"Minimum number of UMI counts based on the knee plot: {min_umi}")
print(f"Mean UMI count per cell (before filtering): {host_adata.X.sum(axis=1).mean()}")
print(f"Max UMI count in one cell: {host_adata.X.sum(axis=1).max()}")
print(f"Total UMI count (before filtering): {host_adata.X.sum()}")

In [ ]:
# Remove cells with less than [min_umi] UMI counts
host_adata = host_adata[np.asarray(host_adata.X.sum(axis=1)).reshape(-1) > min_umi]

# Add the total UMI counts per cell as obs to adata
host_adata.obs['n_counts'] = host_adata.X.sum(axis=1).A1

## Filter for a minimum number of genes
sc.pp.filter_cells(host_adata, min_genes=min_genes)

print(f"Mean UMI count per cell (after filtering): {host_adata.X.sum(axis=1).mean()}")
print(f"Total UMI count (after filtering): {host_adata.X.sum()}\n")

# Filter by mito content

In [ ]:
# Max percentage mitochondrial genes per cell (10%)
mito_filter = 0.1

In [ ]:
# Get macaque mito genes
# Get Ensembl IDs of MT genes
mito_ensembl_df = sc.queries.mitochondrial_genes("mmulatta", attrname="ensembl_gene_id")
macaca_mito_gene_ids = mito_ensembl_df["ensembl_gene_id"].values
macaca_mito_gene_ids

In [ ]:
# Get canine mito genes with gget
gget_df = gget.search(["MT", "mito"], andor="and", species="canis_lupus_familiaris_core_109_1")
gget_df = gget_df[(gget_df["biotype"] == "protein_coding") & (gget_df["gene_name"].str.startswith("MT"))]
canine_mito_gene_ids = gget_df["ensembl_id"].values
canine_mito_gene_ids

In [ ]:
mito_gene_ids = list(macaca_mito_gene_ids) + list(canine_mito_gene_ids)

In [ ]:
# `.A1` transforms sparse array X to a dense array after summing
host_adata.obs['percent_mito'] = np.sum(host_adata[:, mito_gene_ids].X, axis=1).A1 / np.sum(host_adata.X, axis=1).A1
print(f"Maximum % mitochondrially encoded genes in one cell (before filtering): {host_adata.obs.percent_mito.max()}.")
host_adata = host_adata[host_adata.obs.percent_mito < mito_filter]
print(f"Maximum % mitochondrially encoded genes in one cell (after filtering): {host_adata.obs.percent_mito.max()}.")

___
# Separate macaque and MDCK cells

Add species label to genes:

In [ ]:
def label_species(row):
    if row["gene_id"].startswith("ENSCAF"):
        return "canis_lupus_familiaris"
    elif row["gene_id"].startswith("ENSMM"):
        return "macaca_mulatta"
    else:
        return "unknown"

host_adata.var["species"] = host_adata.var.apply(lambda row: label_species(row), axis=1)

In [ ]:
host_adata.var

In [ ]:
host_adata.var["species"].unique()

In [ ]:
host_adata

Plot number of cells with shared gene expression and define cell species based on gene count threshold:

In [ ]:
macaque_count = (np.array(host_adata[:, host_adata.var["species"] == "macaca_mulatta"].X.sum(axis=1))).flatten()

In [ ]:
len(macaque_count[macaque_count > 0])

In [ ]:
canis_count = (np.array(host_adata[:, host_adata.var["species"] == "canis_lupus_familiaris"].X.sum(axis=1))).flatten()

In [ ]:
len(canis_count[canis_count > 0])

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(8, 7))

# Plot number of canis over macaque UMIs
x = macaque_count
y = canis_count

# Histogram data to show point density
bins = [1500, 1500]
hh, locx, locy = np.histogram2d(x, y, bins=bins)
z = np.array([hh[np.argmax(a<=locx[1:]),np.argmax(b<=locy[1:])] for a,b in zip(x,y)])
idx = z.argsort()
x2, y2, z2 = x[idx], y[idx], z[idx]

scatter = ax.scatter(x2, y2, c=z2, cmap="viridis", s=5)

# Add colorbar
fig.colorbar(scatter, ax=ax)

# # Add rectangles showing species filters
# threshold = 20

# p1 = mpatches.Rectangle((0, threshold), threshold, ax.get_ylim()[1] - threshold, alpha=0.1, facecolor="#f77f00")
# ax.add_patch(p1)
# p2 = mpatches.Rectangle((threshold, 0), ax.get_xlim()[1] - threshold, threshold, alpha=0.1, facecolor="green")
# ax.add_patch(p2)

ax.set_title("Total UMI counts for each cell")
ax.set_xlabel("Macaca mulatta UMI count")
ax.set_ylabel("Canis lupus familiaris UMI count")

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

fig.show()

In [ ]:
%%time
# Maximum UMI percentage allowed from other species for species separation
threshold = 0.1

obs_species = []
for mc, cc in zip(macaque_count, canis_count):
    if cc / mc <= threshold:
        obs_species.append("macaca_mulatta")

    elif mc / cc <= threshold:
        obs_species.append("canis_lupus_familiaris")

    else:
        obs_species.append("unkown")

In [ ]:
host_adata.obs["species"] = obs_species

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(12, 7))

# Plot number of canis over macaque UMIs

# Generate colormaps (removing lightest values for visibility)
cmaps_orig = [plt.cm.Reds, plt.cm.Purples, plt.cm.Greys]
min_val, max_val = 0.3, 1.0
n = 10
cmaps = []
for orig_cmap in cmaps_orig:
    colors = orig_cmap(np.linspace(min_val, max_val, n))
    cmaps.append(mpl.colors.LinearSegmentedColormap.from_list("mycmap", colors))

for i, species in enumerate(host_adata.obs["species"].unique()):
    indx = host_adata.obs["species"] == species
    # Plot number of canis over macaque UMIs
    x = macaque_count[indx]
    y = canis_count[indx]

    # Histogram data to show point density
    bins = [1500, 1500]
    hh, locx, locy = np.histogram2d(x, y, bins=bins)
    z = np.array([hh[np.argmax(a<=locx[1:]),np.argmax(b<=locy[1:])] for a,b in zip(x,y)])
    idx = z.argsort()
    x2, y2, z2 = x[idx], y[idx], z[idx]

    scatter = ax.scatter(x2, y2, c=z2, norm=mpl.colors.Normalize(vmin=0, vmax=500), cmap=cmaps[i], s=10)

    # Add colorbar
    cbar = fig.colorbar(scatter, ax=ax, pad=0.01)
    if i == 0:
        cbar.ax.tick_params(axis="both", labelsize=fontsize-2)
    else:
        cbar.set_ticks([])
    if species == "canis_lupus_familiaris":
        cbar.ax.set_ylabel(f"# of {' '.join(species.capitalize().split('_'))} (MDCK) cells")
    else:
        cbar.ax.set_ylabel(f"# of {' '.join(species.capitalize().split('_'))} cells")

# ax.set_title("Total UMI counts for each cell", fontsize=fontsize+2)
ax.set_xlabel("Macaca mulatta UMI count", fontsize=fontsize)
ax.set_ylabel("Canis lupus familiaris UMI count", fontsize=fontsize)

ax.set_xscale("log")
ax.set_yscale("log")

# Change fontsize of tick labels
ax.tick_params(axis="both", labelsize=fontsize)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

# Save figure
fig.savefig(
    f"canis_macaca_umis.png", dpi=300, bbox_inches="tight"
)

fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
width = 0.6

species = ["macaca_mulatta", "canis_lupus_familiaris", "unkown"]
labels = ["Macaca\nmulatta", "MDCK", "Unkown"]

ax.bar(0, len(host_adata.obs[host_adata.obs["species"] == species[0]]), width=width, color="#ae1116")
ax.bar(1, len(host_adata.obs[host_adata.obs["species"] == species[1]]), width=width, color="mediumpurple")
ax.bar(2, len(host_adata.obs[host_adata.obs["species"] == species[2]]), width=width, color="grey")

ax.set_ylabel("Number of cells", fontsize=fontsize)
ax.set_xticks([0,1,2], labels, fontsize=fontsize)

# Change fontsize of tick labels
ax.tick_params(axis="both", labelsize=fontsize)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

# Save figure
fig.savefig(
    f"canis_macaca_umis_bars.png", dpi=300, bbox_inches="tight"
)

fig.show()

___

In [ ]:
# Save filtered AnnData object
host_adata.write("host_QC.h5ad")